In [ ]:
!pip install pymrmR


In [64]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pymrmr

# ========== Load and Prepare Data ==========
data = pd.read_csv("glcm_features.csv")  # Replace with actual path if different
data = data.dropna()
data['Disease %'] = data['Disease %'].astype(float)

# ========== Apply mRMR Feature Selection ==========
mrmr_input = data[['Contrast', 'Correlation', 'Energy', 'Homogeneity', 'ASM', 'Dissimilarity', 'Disease %']]
selected_features = pymrmr.mRMR(mrmr_input, 'MID', 3)  # You can also try 'MID'

# Remove the target label if mistakenly included
if 'Disease %' in selected_features:
    selected_features.remove('Disease %')

print(f"Selected Features by mRMR: {selected_features}")

# ========== Prepare Training Data ==========
X = data[selected_features]
y = data['Disease %']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ========== Train RandomForest Regressor ==========
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ========== Evaluate ==========
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")

# ========== Save Model and Selected Features ==========
joblib.dump(model, "glcm_regressor_mrmr_final.pkl")
joblib.dump(selected_features, "selected_features_mrmr_final.pkl")
print("Model and selected features saved.")


Selected Features by mRMR: ['Dissimilarity', 'Correlation']
Mean Absolute Error: 3.61
R² Score: 0.92
Model and selected features saved.


In [35]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the dataset
data = pd.read_csv("glcm_features.csv")

# Features and label
X = data[['Contrast', 'Correlation', 'Energy', 'Homogeneity', 'ASM', 'Dissimilarity']]
y = data['Disease %']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_scaled)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")

# Save the model and preprocessing objects
joblib.dump(model, "rf_model.pkl")
joblib.dump(pca, "pca.pkl")
joblib.dump(scaler, "scaler.pkl")


Mean Absolute Error: 3.46
R² Score: 0.93


['scaler.pkl']

In [ ]:
!pip install onnxruntime


In [31]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


data = pd.read_csv("glcm_features.csv")

X = data[['Contrast', 'Correlation', 'Energy', 'Homogeneity', 'ASM', 'Dissimilarity']]
y = data['Disease %']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

joblib.dump(model, "glcm_regressor.pkl")
print("Model saved successfully!")


Mean Absolute Error: 3.814846001166073
R² Score: 0.9168747474382969
Model saved successfully!


In [53]:
import rembg
import numpy as np
import cv2
import joblib
from PIL import Image
from skimage.feature import graycomatrix, graycoprops
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_ubyte

def remove_background(image_path):
    input_image = Image.open(image_path).convert("RGB")
    input_array = np.array(input_image)
    output_array = rembg.remove(input_array)
    return Image.fromarray(output_array)

def get_disease_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    color_masks = {
        'yellow': (np.array([22, 120, 100]), np.array([32, 255, 255])),
        'brown': (np.array([8, 100, 40]), np.array([20, 255, 150])),
        'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
        'skin': (np.array([0, 48, 80]), np.array([20, 255, 255]))
    }
    return sum(cv2.inRange(hsv, *color_masks[color]) for color in color_masks)

def extract_glcm_features(image, mask_disease):
    try:
        gray = img_as_ubyte(rgb2gray(image))
        diseased_region = cv2.bitwise_and(gray, gray, mask=mask_disease)

        if diseased_region.max() > 0:
            diseased_region = (diseased_region / diseased_region.max() * 255).astype(np.uint8)

        glcm = graycomatrix(diseased_region, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

        features = [graycoprops(glcm, prop).mean() for prop in ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']]
        return np.array(features)
    except Exception as e:
        print(f"Error extracting GLCM features: {e}")
        return None

def classify_disease(disease_percent):
    if disease_percent > 15:
        return "Confidently Diseased"
    elif 5 <= disease_percent <= 15:
        return "Probably Diseased"
    else:
        return "Confidently Healthy"

def calculate_color_mask_percentage(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    color_masks = {
        'green': (np.array([28, 44, 39]), np.array([75, 255, 255])),
        'yellow': (np.array([22, 120, 100]), np.array([32, 255, 255])),
        'brown': (np.array([8, 100, 40]), np.array([20, 255, 150])),
        'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
        'skin': (np.array([0, 48, 80]), np.array([20, 255, 255]))
    }

    mask_green = cv2.inRange(hsv, *color_masks['green'])
    mask_disease = sum(cv2.inRange(hsv, *color_masks[color]) for color in ['yellow', 'brown', 'orange', 'skin'])

    total_pixels = image.shape[0] * image.shape[1]
    green_pixels = np.sum(mask_green == 255)
    disease_pixels = np.sum(mask_disease == 255)

    if disease_pixels + green_pixels == 0:
        return 0  # Avoid division by zero

    return (disease_pixels / (disease_pixels + green_pixels)) * 100

def predict_disease(image_path, model_path="glcm_regressor.pkl"):
    try:
        model = joblib.load(model_path)
        image = imread(image_path)
        mask_disease = get_disease_mask(image)
        features = extract_glcm_features(image, mask_disease)

        if features is None:
            return "Error extracting features!"

        features = features.reshape(1, -1)
        predicted_disease_percent = model.predict(features)[0]
        mask_disease_percent = calculate_color_mask_percentage(image)

        if predicted_disease_percent < 5 and mask_disease_percent < 5:
            classification = "Confidently Healthy"
        elif (predicted_disease_percent > 5 and mask_disease_percent < 5) or (predicted_disease_percent < 5 and mask_disease_percent > 5):
            classification = "Probably Healthy"
        elif 5 <= predicted_disease_percent <= 15 or 5 <= mask_disease_percent <= 15:
            classification = "Probably Diseased"
        else:
            classification = "Confidently Diseased"

        return predicted_disease_percent, mask_disease_percent, classification
    except Exception as e:
        print(f"Error in disease prediction: {e}")
        return None, None, None

# Example Usage
image_path = r"/content/Fungi.jpg"
disease_percent_glcm, disease_percent_mask, classification = predict_disease(image_path)

if disease_percent_glcm is not None:
    print(f"Predicted Disease % (GLCM Model): {disease_percent_glcm:.2f}%")
    print(f"Predicted Disease % (Color Masking): {disease_percent_mask:.2f}%")
    print(f"Final Classification: {classification}")
else:
    print("Prediction failed due to an error.")


Predicted Disease % (GLCM Model): 15.83%
Predicted Disease % (Color Masking): 9.07%
Final Classification: Probably Diseased


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [52]:
import rembg
import numpy as np
import cv2
import joblib
from PIL import Image
from skimage.feature import graycomatrix, graycoprops
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_ubyte

# Load saved model and preprocessing steps
model = joblib.load("rf_model.pkl")
scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")

def remove_background(image_path):
    input_image = Image.open(image_path).convert("RGB")
    input_array = np.array(input_image)
    output_array = rembg.remove(input_array)
    return Image.fromarray(output_array)

def get_disease_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    color_masks = {
        'yellow': (np.array([22, 120, 100]), np.array([32, 255, 255])),
        'brown': (np.array([8, 100, 40]), np.array([20, 255, 150])),
        'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
        'skin': (np.array([0, 48, 80]), np.array([20, 255, 255]))
    }
    return sum(cv2.inRange(hsv, *color_masks[color]) for color in color_masks)

def extract_glcm_features(image, mask_disease):
    try:
        gray = img_as_ubyte(rgb2gray(image))
        diseased_region = cv2.bitwise_and(gray, gray, mask=mask_disease)

        if diseased_region.max() > 0:
            diseased_region = (diseased_region / diseased_region.max() * 255).astype(np.uint8)

        glcm = graycomatrix(diseased_region, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                            levels=256, symmetric=True, normed=True)

        features = [graycoprops(glcm, prop).mean()
                    for prop in ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']]
        return np.array(features)
    except Exception as e:
        print(f"Error extracting GLCM features: {e}")
        return None

def calculate_color_mask_percentage(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    color_masks = {
        'green': (np.array([28, 44, 39]), np.array([75, 255, 255])),
        'yellow': (np.array([22, 120, 100]), np.array([32, 255, 255])),
        'brown': (np.array([8, 100, 40]), np.array([20, 255, 150])),
        'orange': (np.array([10, 50, 50]), np.array([20, 255, 255])),
        'skin': (np.array([0, 48, 80]), np.array([20, 255, 255]))
    }

    mask_green = cv2.inRange(hsv, *color_masks['green'])
    mask_disease = sum(cv2.inRange(hsv, *color_masks[color]) for color in ['yellow', 'brown', 'orange', 'skin'])

    total_pixels = image.shape[0] * image.shape[1]
    green_pixels = np.sum(mask_green == 255)
    disease_pixels = np.sum(mask_disease == 255)

    if disease_pixels + green_pixels == 0:
        return 0  # Avoid division by zero

    return (disease_pixels / (disease_pixels + green_pixels)) * 100

def predict_disease(image_path):
    try:
        image = imread(image_path)
        mask_disease = get_disease_mask(image)
        features = extract_glcm_features(image, mask_disease)

        if features is None:
            return "Error extracting features", None, None

        features_scaled = scaler.transform(features.reshape(1, -1))
        features_pca = pca.transform(features_scaled)

        predicted_disease_percent = model.predict(features_pca)[0]
        mask_disease_percent = calculate_color_mask_percentage(image)

        if predicted_disease_percent < 5 and mask_disease_percent < 5:
            classification = "Confidently Healthy"
        elif (predicted_disease_percent > 5 and mask_disease_percent < 5) or \
             (predicted_disease_percent < 5 and mask_disease_percent > 5):
            classification = "Probably Healthy"
        elif 5 <= predicted_disease_percent <= 15 or 5 <= mask_disease_percent <= 15:
            classification = "Probably Diseased"
        else:
            classification = "Confidently Diseased"

        return predicted_disease_percent, mask_disease_percent, classification
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None, None

# Example usage
if __name__ == "__main__":
    image_path = "/content/Fungi.jpg"  # Replace with your test image path
    glcm_percent, mask_percent, result = predict_disease(image_path)

    if glcm_percent is not None:
        print(f"Predicted Disease % (GLCM): {glcm_percent:.2f}")
        print(f"Predicted Disease % (Mask): {mask_percent:.2f}")
        print(f"Classification: {result}")
    else:
        print("Prediction failed.")


Predicted Disease % (GLCM): 16.29
Predicted Disease % (Mask): 9.07
Classification: Probably Diseased


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [66]:
import numpy as np
import cv2
import joblib
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage import img_as_ubyte

# ========== Load Model and Selected Features ==========
model = joblib.load("glcm_regressor_mrmr_final.pkl")
selected_features = joblib.load("selected_features_mrmr_final.pkl")

# ========== Helper Functions ==========

def get_disease_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = sum(cv2.inRange(hsv, *range_) for range_ in [
        (np.array([22, 120, 100]), np.array([32, 255, 255])),  # yellow
        (np.array([8, 100, 40]), np.array([20, 255, 150])),    # brown
        (np.array([10, 50, 50]), np.array([20, 255, 255])),    # orange
        (np.array([0, 48, 80]), np.array([20, 255, 255]))      # skin-like
    ])
    return mask

def extract_mrmr_glcm_features(image, mask_disease, selected_features):
    try:
        gray = img_as_ubyte(rgb2gray(image))
        diseased_region = cv2.bitwise_and(gray, gray, mask=mask_disease)

        if diseased_region.max() > 0:
            diseased_region = (diseased_region / diseased_region.max() * 255).astype(np.uint8)

        glcm = graycomatrix(diseased_region, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                            levels=256, symmetric=True, normed=True)

        all_props = {
            'Contrast': graycoprops(glcm, 'contrast').mean(),
            'Correlation': graycoprops(glcm, 'correlation').mean(),
            'Energy': graycoprops(glcm, 'energy').mean(),
            'Homogeneity': graycoprops(glcm, 'homogeneity').mean(),
            'ASM': graycoprops(glcm, 'ASM').mean(),
            'Dissimilarity': graycoprops(glcm, 'dissimilarity').mean(),
        }

        feature_vector = np.array([all_props[feat] for feat in selected_features]).reshape(1, -1)
        return feature_vector
    except Exception as e:
        print(f"Error extracting GLCM features: {e}")
        return None

def calculate_disease_percent_by_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    green_range = (np.array([28, 44, 39]), np.array([75, 255, 255]))
    green_mask = cv2.inRange(hsv, *green_range)

    disease_masks = [
        (np.array([22, 120, 100]), np.array([32, 255, 255])),
        (np.array([8, 100, 40]), np.array([20, 255, 150])),
        (np.array([10, 50, 50]), np.array([20, 255, 255])),
        (np.array([0, 48, 80]), np.array([20, 255, 255]))
    ]
    disease_mask = sum(cv2.inRange(hsv, low, high) for (low, high) in disease_masks)

    green_pixels = np.sum(green_mask == 255)
    disease_pixels = np.sum(disease_mask == 255)

    total_affected = green_pixels + disease_pixels
    if total_affected == 0:
        return 0

    return (disease_pixels / total_affected) * 100

def predict_disease_percent(image_path):
    image = imread(image_path)
    mask = get_disease_mask(image)

    features = extract_mrmr_glcm_features(image, mask, selected_features)
    if features is None:
        return "Feature extraction failed", None, None

    prediction = model.predict(features)[0]
    mask_disease_percent = calculate_disease_percent_by_mask(image)

    if prediction < 5 and mask_disease_percent < 5:
        label = "Confidently Healthy"
    elif (prediction > 5 and mask_disease_percent < 5) or (prediction < 5 and mask_disease_percent > 5):
        label = "Probably Healthy"
    elif 5 <= prediction <= 15 or 5 <= mask_disease_percent <= 15:
        label = "Probably Diseased"
    else:
        label = "Confidently Diseased"

    return prediction, mask_disease_percent, label

# ========== Example Test ==========
if __name__ == "__main__":
    image_path = "/content/Fungi.jpg"  # Replace with your image file
    glcm_pred, mask_pred, status = predict_disease_percent(image_path)

    if glcm_pred is not None:
        print(f"GLCM Predicted Disease %: {glcm_pred:.2f}")
        print(f"Mask-based Disease %: {mask_pred:.2f}")
        print(f"Classification: {status}")
    else:
        print("Prediction failed.")


GLCM Predicted Disease %: 13.52
Mask-based Disease %: 9.07
Classification: Probably Diseased


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
